In [1]:
import configparser
import os

In [2]:
# get user inputs from config file:

def join_path(base, filename):
    if base.startswith('http'):
        return f"{base.rstrip('/')}/{filename.lstrip('/')}"
    else:
        return os.path.join(base, filename)

config_file = os.path.abspath("config.ini")
config = configparser.ConfigParser()
config.read(config_file)

input_base = config['global']['input_base']
process_base = config['global']['process_base']
output_base = config['global']['output_base']

input_dict = dict(config['input'])
process_dict = dict(config['process'])
output_dict = dict(config['output'])

In [3]:
##################################
# Read your input data from config file
##################################
aoi = join_path(input_base, input_dict['aoi'])
## Accessibility
road = join_path(input_base, input_dict['road'])
settlement = join_path(input_base, input_dict['settlement'])
river = join_path(input_base, input_dict['river'])

## Topography
elevation = join_path(input_base, input_dict['elevation'])
slope = join_path(input_base, input_dict['slope'])

## Forest or deforestation history
forest_cover_t1 = join_path(input_base, input_dict['forest_cover_t1'])
forest_cover_t2 = join_path(input_base, input_dict['forest_cover_t2'])
forest_cover_t3 = join_path(input_base, input_dict['forest_cover_t3'])

## Land conservation status
protected_area = join_path(input_base, input_dict['protected_area'])

In [4]:
# reload all modules automatically when changes happen
%load_ext autoreload
%autoreload 2

In [5]:
##################################
# Processing data. Read from process config file
##################################

road_process = join_path(process_base, process_dict['road'])
settlement_process = join_path(process_base, process_dict['settlement'])
forest_cover_change_t12 = join_path(process_base, process_dict['forest_cover_change_t12'])
forest_cover_change_t23 = join_path(process_base, process_dict['forest_cover_change_t23'])

distance_deforestation_t12 = join_path(process_base, process_dict['distance_deforestation_t12'])
distance_deforestation_t23 = join_path(process_base, process_dict['distance_deforestation_t23'])

distance_road = join_path(process_base, process_dict['distance_road'])
distance_river = join_path(process_base, process_dict['distance_river'])

In [6]:
from models import ProcessingForecastDeforestation

process = ProcessingForecastDeforestation(target_crs=32749)


==================== BEGINNING LOG ====================


In [8]:
##################################
# Run the process. Initialize the input model
##################################

%time
process.clip_raster_to_aoi(settlement, aoi, settlement_process)
process.rasterize(road, road_process)
process.compute_deforestation(forest_cover_t1, forest_cover_t2, forest_cover_change_t12, nodata=None)
process.compute_deforestation(forest_cover_t2, forest_cover_t3, forest_cover_change_t23, nodata=None)

CPU times: total: 0 ns
Wall time: 0 ns


Clipped raster written: process\settlement.tif
Process took 8.44 seconds
Raster written: process\road.tif (12154x3685, pixel size: 30 units)
Process took 14.30 seconds
Deforestation raster saved to process\forest_cover_change_t12.tif
Process took 22.16 seconds
Deforestation raster saved to process\forest_cover_change_t23.tif
Process took 10.67 seconds


'process\\forest_cover_change_t23.tif'

### Compute the Euclidean Distance as a means of estimating the probability of deforestation

#### Read the paper refence here: https://joss.theoj.org/papers/10.21105/joss.02975

In [ ]:
%time

process.euclidean_distance(road_process, distance_road, 100)
process.euclidean_distance(forest_cover_change_t12, distance_deforestation_t12, 100)
process.euclidean_distance(forest_cover_change_t23, distance_deforestation_t23, 100)

CPU times: total: 0 ns
Wall time: 0 ns


Euclidean distance raster saved as process\distance_road.tif
Process took 8.57 seconds
